In [4]:
import numpy as np
import matplotlib.pyplot as plt
from data.mnist import load_mnist

### Optimizaition Methods
![7](https://user-images.githubusercontent.com/38183218/44017435-440dd208-9f13-11e8-8d7c-0fc626274bc8.PNG)

![1](https://user-images.githubusercontent.com/38183218/44017423-41111d30-9f13-11e8-9181-b582c4973096.PNG)

#### Stochastic Gradient Descent(full batch와 비교)
- 배치마다 gradient 내려가는 모양다르기에 local minima에 빠지는 위험 피할수있음



- 그러나

![1 5](https://user-images.githubusercontent.com/38183218/44017424-413a8de6-9f13-11e8-9117-65d2c48b8d4c.PNG)
![2](https://user-images.githubusercontent.com/38183218/44017425-41632972-9f13-11e8-861d-fc9ad89036b7.PNG)

- SGD의 단점은 비등방성함수(방향에 따라 성질, 즉 기울기가 달라지는 함수)에서 탐색 경로가 비효율적
![](https://user-images.githubusercontent.com/38183218/45130977-e3017f80-b1c5-11e8-8269-27574584cf26.png)
- 예들어 위와같이 y축방향 기울기 가파르지만 x축방향 기울기는 완만한경우. 최적점인 (0,0)에 도달하는 시간이 매우 오래걸림

- 근본적인 한계도 존재, 그레디언트의 방향이 최적점과 다른방향을 가리키는 문제

#### Momentum Update
같은 방향이면 쭉 밀어주기
![3](https://user-images.githubusercontent.com/38183218/44017426-418b4b64-9f13-11e8-92cb-3ec0ca5b61b7.PNG)

In [2]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
            self.lr = lr
            self.momentum = momentum
            self.v = None
            
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zero_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key] 

#### Adagrad Update
노드마다 lr다르게 걸어서 학습률 최적화. 보통 학습률을 서서히 낮추는 형태. 

직관적으로 생각해도 최적점찾으려면 처음에는 보폭을 크게, 나중에는 작게해야.

큰 움직임에 비례해 갱신 정도도 큰폭으로 작아진다.

그래서 SGD와 같이 지그재그 형태로 움직이더라도 y축방향으로 갱신강도가 빠르게 약해지고, 지그재그의 움직임이 줄어든다

cache값이 점점 증가하므로 어느순간 dx가 0 되는 단점 -> RMSProp으로 해결

![4](https://user-images.githubusercontent.com/38183218/44017432-439170aa-9f13-11e8-869d-bddbaea10c9b.PNG)



#### RMSProp Update
기울기를 제곱하여 계속 더해가는 AdaGrad의 cache, 그래서 학습 진행할수록 갱신강도가 약해진다

실제로 어느순간 갱신량이 0이된다

RMSProp은 지수이동평균(EMA)을 사용하여 과거 기울기의 반영 정도를 기하급수적으로 감소시킨다

decay rate를 사용
![5](https://user-images.githubusercontent.com/38183218/44017433-43bab082-9f13-11e8-9fb1-ad410728df20.PNG)


#### Adam Update

Adam Optimization 논문 요약/정리 : http://dalpo0814.tistory.com/29

모멘텀 + RMSprop


In [5]:
# Adam 간단히 구현
m = beta1*m + (1-beta1)*dx 
v = beta2*v + (1-beta2)*(dx**2) 
x -= learning_Rate * m/(np.sqrt(v) + 1e-7)

In [3]:
class Adam:

    """Adam (http://arxiv.org/abs/1412.6980v8)"""

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            #self.m[key] = self.beta1*self.m[key] + (1-self.beta1)*grads[key]
            #self.v[key] = self.beta2*self.v[key] + (1-self.beta2)*(grads[key]**2)
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
            
            #unbias_m += (1 - self.beta1) * (grads[key] - self.m[key]) # correct bias
            #unbisa_b += (1 - self.beta2) * (grads[key]*grads[key] - self.v[key]) # correct bias
            #params[key] += self.lr * unbias_m / (np.sqrt(unbisa_b) + 1e-7)

![](https://user-images.githubusercontent.com/38183218/45130955-e0068f00-b1c5-11e8-8a4e-cfb1d55eb9cb.png)

### Initial Weights

__weight decay__ 라는것이 필요 : 가중치 값을 작게하여 오버피팅을 억제하는 방법

#### 그렇다면 Initial Weights를 0으로하면 어떨까?
- 학습이 안됨
- 정확히말하면 가중치를 균일한 값으로 설정하면 안됨, 모든 가중치가 똑같이 갱신됨
- 대칭적인 구조로 이루어지는 역전파(곱셈, 덧셈노드)이기에 대칭구조를 깰려면 가중치의 초깃값을 무작위로 설정해야한다

#### activation값의 분포
- 표준편차 1인 정규분포로 초기화한 가중치의 activation값
![](https://user-images.githubusercontent.com/38183218/45130958-e0068f00-b1c5-11e8-8f4b-4f70fcc5940d.png)

- 시그모이드 함수의 경우 출력이 0 또는 1에 가까워질수록 미분이 0에 다가감.
- -> __gradient vanishing__ 의 문제

- 반면 표준편차 0.01인 정규분포로 초기화한 가중치의 activation값
- 0.5부근에 집중되어 gradient vanishing은 일어나지 않음
- 그러나 대부분의 뉴런이 같은값을 출력하므로 __표현력이 제한__ 되는 문제
![](https://user-images.githubusercontent.com/38183218/45130959-e09f2580-b1c5-11e8-9c4f-ed5e59bb2403.png)

- __Xavier 초깃값__
- 노드개수가 n일때 표준편차가 1/sqrt(n)인 정규분포로 가중치를 초기화
![](https://user-images.githubusercontent.com/38183218/45130961-e09f2580-b1c5-11e8-9ccb-eb641421e6da.png)

#### ReLu 사용시의 가중치 초깃값 
- He 초깃값 -> 표준편차로 sqrt(2/n)
- Xavier 초깃값은 활성화함수가 선형이거나 선형과 유사한 성질(sigmoid, tanh는 중앙부근이 선형)일때만 사용
![](https://user-images.githubusercontent.com/38183218/45130962-e09f2580-b1c5-11e8-92ae-08607dc21141.png)

- He 초깃값인 경우 모든 층에서 활성화값이 균일하게 분포
- 결론적으로 ReLu일땐 He, sigmoid나 tanh일땐 Xavier

### Batch Normalization
원활한 학습위해선 활성화값의 분포가 적당히 퍼져있어야함

그렇다면 각 층이 활성화값을 적당히 퍼뜨리도록 '강제'하면 어떠냐

-> Batch Normalization의 아이디어

즉, 레이어가 쌓일 수록 이상치인 batch가 가져오는 오차는 커지니까

그래서 애초에 각층마다 배치정규화 계층넣어서 zero-centered로 normalize해버리자

- 학습속도개선
- 초기값에 크게 의존x
- 오버피팅 억제해 드롭아웃 등의 필요성 감소

![](https://user-images.githubusercontent.com/38183218/45130964-e137bc00-b1c5-11e8-8b55-09e6941e022d.png)
배치정규화 계층을 신경망에 삽입

미니배치단위로 정규화(평균 0, 분산1의 표준화까지)

결과로 빠른 학습속도
![](https://user-images.githubusercontent.com/38183218/45130967-e1d05280-b1c5-11e8-880d-cc823036469c.png)



### Regularization
오버피팅 방지 위함

__오버피팅 발생 가능성__
- 매개변수가 많고 표현력이 높은 모델
- 훈련데이터가 적은 모델

오버피팅 발생시
![](https://user-images.githubusercontent.com/38183218/45130971-e268e900-b1c5-11e8-8d1a-fb379302f5bd.png)

#### 가중치 감소(weight decay)
- 전통적인 오버피팅 억제방법
- 학습 과정에서 큰 가중치에 대해 큰 페널티 부과
- 가중치 매개변수의 값이 커서 발생하는 오버피팅이 많은데 이를 해결

![](https://user-images.githubusercontent.com/38183218/45263987-9e007600-b46f-11e8-9dec-0a305e9a529d.PNG)

가중치 각각의 손실 함수에 1/2 \* lambda \* $W^2$ 을 더한다

따라서 역전파 시에 정규화 항을 미분한 lambda\*W 항이 더해진다

아래 그래프는 lambda =0.1로 가중치 감소 적용한 결과

-> 오버피팅 정도가 완화
![](https://user-images.githubusercontent.com/38183218/45130972-e268e900-b1c5-11e8-9978-d288dfdb4514.png)

#### 드롭아웃
- 복잡한 신경망에는 가중치 감소만으로 대응하기 어려움
- 그럴땐 뉴런을 임의로 삭제하며 학습하는 방법인 드롭아웃을 이용한다
- 학습 중에 hidden layer 의 뉴런을 무작위로 삭제하여 신호 전달을 못하게 한다


![](https://user-images.githubusercontent.com/38183218/45130974-e268e900-b1c5-11e8-8c40-2025c72e1ceb.png)

오른쪽이 dropout_ratio = 0.15로 드롭아웃을 적용한 결과이다. 오버피팅이 억제되었다
![](https://user-images.githubusercontent.com/38183218/45130975-e3017f80-b1c5-11e8-97f0-a11845c696f6.png)

#### Ensembles
- 일반적인 기계학습에서의 앙상블과 같이 개별적으로 학습시킨 여러 신경망의 출력을 평균(혹은 voting)하여 추론한다

- 예를 들면 같은(혹은 비슷한) 구조의 신경망 m개를 준비하여 개별적으로 학습시키고, 테스트 시에 m개 출력의 평균을 내어 답하는 것

- 드롭아웃과 밀접한 관련이 있는데, 드롭아웃이 뉴런을 무작위 삭제하는 행위를 매번 다른 모델을 학습시키는 것으로 해석할 수 있기 때문
![](https://user-images.githubusercontent.com/38183218/45263988-9e990c80-b46f-11e8-9d69-ee0e8987ea41.PNG)

#### Data Augmentation
![](https://user-images.githubusercontent.com/38183218/45263989-9e990c80-b46f-11e8-80cc-e16068f268df.PNG)

Ramdom mix of 
- translation
- rotation
- stretching
- shearing
- lens distortion...